In [1]:
%load_ext autotime

# Postage Stamp Cube (PSC) Creation

For each of the sources that were detected we want to slice out the Postage Stamp Cube (PSC). The _x_ and _y_ values detected from `sextractor` are used to located the star, however this position is then adjusted such that every stamp in the cube is aligned and has a red-pixel in the (0,0) position. The _x,y_ values for each frame should lie within the center superpixel on the corresponding stamp.

The stamps contain the raw data, i.e. no bias removed or any other processing. 

Each stamp is saved along with the timestamp and picid inside a csv file, one file per PSC.

In [2]:
import os

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook as tqdm

from piaa.utils import pipeline
from piaa.utils import helpers

from astropy.io import fits

time: 2.95 s


In [3]:
fields_dir = os.path.join(os.environ['PANDIR'], 'images', 'fields')

time: 1.25 ms


In [4]:
# base_dir = os.path.join(fields_dir, 'PAN001/Hd189733/14d3bd/20180913T085704')
base_dir = os.path.join(fields_dir, 'PAN012/Hd189733/358d0f/20180822T035809')
# base_dir = os.path.join(fields_dir, 'PAN001/Wasp50/14d3bd/20181215T050712')

base_dir

'/var/panoptes/images/fields/PAN012/Hd189733/358d0f/20180822T035809'

time: 13.8 ms


In [5]:
# Get detected sources
source_filename = os.path.join(base_dir, f'point-sources-filtered.csv.bz2')

# Actually get the sources
sources = pipeline.lookup_sources_for_observation(filename=source_filename).set_index(['picid'], append=True)

# Make directory for PSC
os.makedirs(os.path.join(base_dir, 'stamps'), exist_ok=True)

time: 1.55 s


In [6]:
# Used for progress display
num_sources = len(list(sources.index.levels[1].unique()))
num_sources

334

time: 8.33 ms


In [7]:
stamp_size = 10

time: 805 µs


In [8]:
picid = None
# picid = 256364928
# picid = 382391899  # Wasp 50

time: 4.63 ms


In [9]:
# x = sources.query(f'picid == {picid}').x
# y = sources.query(f'picid == {picid}').y

# x = x - x.mean()
# y = y - y.mean()

time: 593 µs


In [10]:
for pid, target_table in tqdm(sources.groupby('picid'), total=num_sources, desc="Making PSCs"):
    stamps = list()
    
    if picid and pid != picid:
        continue
    
    for idx, row in target_table.iterrows():
        date_obs= idx[0]
        
        # Get the data for the entire frame
        data = fits.getdata(os.path.join(base_dir, row.file)) 

        # Get the stamp for the target
        target_slice = helpers.get_stamp_slice(
            row.x, row.y, 
            stamp_size=(stamp_size, stamp_size), 
            ignore_superpixel=False,
            verbose=False
        )

        # Get data
        stamps.append(data[target_slice].flatten())
        
    pd.DataFrame(stamps, index=target_table.index).to_csv(os.path.join(base_dir, 'stamps', f'{pid}.csv'))


time: 50min 53s
